In [4]:
import tifffile as tf
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.signal import savgol_filter, find_peaks
import os


In [14]:
# choose video

VID_DIRs = {'260pM':'/media/marcus/9C33-6BBD/20210920_GramA_260pM07_3nM815_1mMKCl_1/260pM/',
            '2.6nM': '/media/marcus/9C33-6BBD/20210920_GramA_260pM07_3nM815_1mMKCl_1/3nM/' 
           }


vids ={'260pM':'20210920_GramA_260pM07_3nM815_1mMKCl_1_MMStack_Pos3.ome.tif'
       , '2.6nM': '20210920_GramA_260pM07_3nM815_1mMKCl_1_MMStack_Pos9.ome.tif' }


#smoothing

smoothing = 17
grad_smooth = 81


# set ROI for background

rois = {}
       
#rois in format [x_left,y_top,x_right,y_bottom]
       
rois['260pM']=[149,4,314,45]
rois['2.6nM']=[357,367,459,486]

t_start = {'260pM':45 , '2.6nM':30}
       

dts = {key:0.25 for key in VID_DIRs.keys()}

# peak finding

rel_peak_height = 0.5
distance_between_peaks = 10

In [10]:
rois

{'260pM': [149, 4, 314, 45], '2.6nM': [357, 367, 459, 486]}

In [15]:
backgrounds = {}
grads = {}
peaks = {}
ts = {}
for key in vids.keys():
    
    video = tf.TiffFile(os.path.join(VID_DIRs[key],vids[key]))

    frames = video.asarray()

    x_left = rois[key][0]
    y_top = rois[key][1]
    x_right = rois[key][2]
    y_bottom = rois[key][3]


    roi = frames[t_start[key]:,y_top:y_bottom,x_left:x_right]

    background = np.median(roi,axis = (1,2))
    background = savgol_filter(background,smoothing,1)
    background = background - np.mean(background[:t_start[key]])
    background = background/np.max(background)
    
    backgrounds[key] = background
    
    grad = np.gradient(background)
    grad = savgol_filter(grad,grad_smooth,1)
    grad = grad/np.max(grad)

    grads[key] = grad
    
    peak = find_peaks(np.absolute(grad),height=rel_peak_height,distance=distance_between_peaks)
    peak = peak[0]
    peak = np.append(peak,len(background))

    peaks[key] = peak
    t = dts[key]*np.arange(len(background))
    ts[key]= t

OME series: not an ome-tiff master file
OME series: not an ome-tiff master file


In [16]:
peaks

{'260pM': array([160, 374, 866]), '2.6nM': array([179, 262, 881])}

In [22]:
%matplotlib qt
fig,ax = plt.subplots(2,1)

for i,key in enumerate(backgrounds.keys()):

    t = ts[key]
    background = backgrounds[key]
    peak = peaks[key]
    ax[i].plot(t,background)

    for j in range(1,len(peak)):
        ax[i].fill_between(dts[key]*np.arange(peak[j-1],peak[j]),np.zeros((peak[j]-peak[j-1],)),background[peak[j-1]:peak[j]],linestyles = 'dashed',alpha = 0.25)

    ax[i].tick_params(direction = 'in',left = True,right = True,bottom = True, top = True,length = 7)

    x_ticks = [0,100,200]

    y_ticks = [0,0.5,1]

    ax[i].set_xticks(x_ticks)
    ax[i].set_xticklabels(x_ticks,fontsize = 15)
    ax[i].set_yticks(y_ticks)
    ax[i].set_yticklabels(y_ticks,fontsize = 15)

    ax[i].set_ylabel('Rel I. [a.u]',fontsize = 18)
    ax[i].set_xlabel('Time [mins]',fontsize = 18)
